<a href="https://colab.research.google.com/github/SiyiGit/github_actions/blob/main/CNN_model%2004152024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#install opendatasets
!pip install opendatasets

In [2]:
#import libraries
import opendatasets as od
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.image import imread
import cv2
import random
import os
from os import listdir
from PIL import Image
import seaborn as sns

In [3]:
#kaggle dataset download
od.download("https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: alicjasosialuk
Your Kaggle Key: ··········


100%|██████████| 2.70G/2.70G [00:27<00:00, 107MB/s] 


In [ ]:
#just for info{"username":"alicjasosialuk","key":"fa69bfa4724b0a10006ba687db303759"}

In [4]:
#path to datasets
data_dir_train = './new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'
data_dir_val = './new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'
data_dir_test = './new-plant-diseases-dataset/test/test'

In [5]:
## Defining batch specfications
batch_size = 200
img_height = 100
img_width = 100

In [17]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, MaxPooling2D, Conv2D, Activation, BatchNormalization
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

from sklearn import metrics

In [7]:
# Loading training set
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                  zoom_range=0.2,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  fill_mode='nearest')
train_data = train_datagen.flow_from_directory(data_dir_train,
                                              batch_size=batch_size,
                                              target_size=(img_height, img_width),
                                              class_mode = 'categorical',
                                              shuffle=True)

Found 70295 images belonging to 38 classes.


In [8]:
## loading validation set
valdata_generator=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
val_data = valdata_generator.flow_from_directory(data_dir_val,
                                            batch_size=batch_size,
                                            target_size=(img_height, img_width),
                                            class_mode = 'categorical',
                                            shuffle=False)

Found 17572 images belonging to 38 classes.


In [9]:
## Defining model layers
#inputs = Input(shape = (256, 256,3), name = "Input")

#first_layer = Conv2D(filters = 32,
#                     kernel_size = (5, 5),
#                     padding = 'valid',
#                     activation = 'relu')

#second_layer = MaxPooling2D(pool_size = (2, 2))

#third_layer = Dropout(rate = 0.2)

#fourth_layer = Flatten()

#fifth_layer = Dense(units = 128,
#                    activation = 'relu')

#output_layer = Dense(units = 38,
                     activation='softmax')

#x=first_layer(inputs)
#x=second_layer(x)
#x=third_layer(x)
#x=fourth_layer(x)
#x=fifth_layer(x)

#outputs=output_layer(x)

#cnn_model = Model(inputs = inputs, outputs = outputs)



In [18]:
def create_model():
    model = Sequential([
        Conv2D(filters=128, kernel_size=(5, 5), padding='valid', input_shape=(img_width, img_height, 3)),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),

        Conv2D(filters=64, kernel_size=(3, 3), padding='valid', kernel_regularizer=l2(0.00005)),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),

        Conv2D(filters=32, kernel_size=(3, 3), padding='valid', kernel_regularizer=l2(0.00005)),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),

        Flatten(),

        Dense(units=256, activation='relu'),
        Dropout(0.5),
        Dense(units=128, activation='relu'),
        Dropout(0.5),
        Dense(units=38, activation='sigmoid')
    ])

    return model

In [19]:
cnn_model = create_model()

In [20]:
cnn_model.compile(loss='categorical_crossentropy', # loss function
                  optimizer='adam',                # gradien descent algorithm
                  metrics=['accuracy'])           # accuracy metric)


In [21]:
print(cnn_model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 96, 96, 128)       9728      
                                                                 
 activation (Activation)     (None, 96, 96, 128)       0         
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 48, 48, 128)       0         
 g2D)                                                            
                                                                 
 batch_normalization (Batch  (None, 48, 48, 128)       512       
 Normalization)                                                  
                                                                 
 conv2d_2 (Conv2D)           (None, 46, 46, 64)        73792     
                                                                 
 activation_1 (Activation)   (None, 46, 46, 64)        0

In [23]:
training_history = cnn_model.fit(val_data,
                                 validation_data= val_data,
                                 epochs = 10,
                                 batch_size = 100)

Epoch 1/10
88/88 [==============================] - 1396s 16s/step - loss: 3.8452 - accuracy: 0.0455 - val_loss: 3.5622 - val_accuracy: 0.0484
Epoch 2/10
88/88 [==============================] - 1409s 16s/step - loss: 3.5403 - accuracy: 0.0670 - val_loss: 3.8195 - val_accuracy: 0.0280
Epoch 3/10
88/88 [==============================] - 1461s 17s/step - loss: 3.4688 - accuracy: 0.0852 - val_loss: 4.0355 - val_accuracy: 0.0411
Epoch 4/10
88/88 [==============================] - 1404s 16s/step - loss: 3.3715 - accuracy: 0.1180 - val_loss: 3.3004 - val_accuracy: 0.1062
Epoch 5/10
88/88 [==============================] - 1398s 16s/step - loss: 3.2568 - accuracy: 0.1340 - val_loss: 3.3477 - val_accuracy: 0.1113
Epoch 6/10
88/88 [==============================] - 1380s 16s/step - loss: 3.1596 - accuracy: 0.1558 - val_loss: 3.2898 - val_accuracy: 0.1049
Epoch 7/10
88/88 [==============================] - 1419s 16s/step - loss: 3.0552 - accuracy: 0.1788 - val_loss: 2.8686 - val_accuracy: 0.2524